# Try Fastai

Fastai provides an API for semantic segmentation. The first attempt is to learn how to use it.

In [ ]:
from fastai import *
from fastai.vision import *
from fastai.callbacks.hooks import *

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image

from pathlib import Path
import os
import random

from fastai.callbacks.hooks import *
from fastai.utils.mem import *

## Preparation

Main challenge with fastai is the creation of databunch. After that its all straightforward.

In [ ]:
PROJ_DIR = Path(os.path.abspath("")).parent
PROJ_DIR

In [ ]:
path_img = PROJ_DIR / "data" / "raw" / "train_images"
path_test_img = PROJ_DIR / "data" / "raw" / "test_images"

In [ ]:
fnames = get_image_files(path_img)
fnames[:3]

In [ ]:
img_f = fnames[10]
img = open_image(img_f)
img.show(figsize=(5,5))

In [ ]:
img = open_image(path_img / '0011165.jpg')
img.show(figsize=(5,5))

### Reading Labels

In [ ]:
cloud_class = ["Fish", "Flower", "Gravel", "Sugar"]
cloud_class_code = {x: cloud_class.index(x) + 1 for x in cloud_class}
cloud_class_code

`rle_decode` method of fastai can decome runnling label encodings.

In [ ]:
df_labels = pd.read_csv(PROJ_DIR / "data" / "raw" / "train.csv")
df_labels.head()

In [ ]:
def rle_2_png(img):
    """
    Takes the name of the image, creates the mask will all
    labels available and saves the mask as png
    """
    # If img lacks 'jpg' extension add it
    if not img.endswith("jpg"):
        img += ".jpg"

    # Filter labels dataframe
    df_temp = df_labels[df_labels.Image_Label.apply(lambda x: img in x)]

    # Drop rows with missing lables
    df_temp = df_temp[~df_temp.EncodedPixels.isna()]

    if len(df_temp) == 0:
        raise ValueError(f"{img} has no labels existing.")

    # Create combined mask for each class of clouds
    mask_complete = np.zeros((2100, 1400))
    for ind in range(len(df_temp)):
        cloud_cls = df_temp.iloc[ind].Image_Label.split("_")[-1]

        mask = rle_decode(df_temp.iloc[ind].EncodedPixels, (2100, 1400))

        mask_complete[mask != 0] = cloud_class_code[cloud_cls]
    
    return mask_complete

    plt.imshow(mask_complete.T)
    plt.savefig(PROJ_DIR / "data" / "raw" / "train_labels" / f"{img.split('.')[0]}.png")

### Save the masks in train_labels folder

In [ ]:
for ind, fname in enumerate(fnames):
    if ind % 200 == 0:
        print(ind)
    m = rle_2_png(fname.stem)
    break
    

## Create DataBunch

In [ ]:
x = (SegmentationItemList
     .from_folder(path_img)
     .split_by_rand_pct())
x